Nisarg Patel  
CSC 578- NN and Deep Learning

Problem Statement : The task of the competition is to predict the temperature at the next hour given the weather conditions and temperature for prior hours.  This is a multivariate timeseries forecasting problem, and we use a neural network approach to tackle the problem.

In [ ]:
import numpy as np 
import pandas as pd
import os
import tensorflow as tf
from matplotlib import pyplot
import matplotlib.pyplot as plt
%matplotlib inline
import keras
keras.__version__
pd.set_option('display.max_rows',50)

In [ ]:
dataset = pd.read_csv("../input/climate_hour.csv") #,header=0, index_col=0

In [ ]:
dataset.head()

In [ ]:
dataset.shape

In [ ]:
submission = pd.read_csv('../input/sample_submission.csv')
submission = submission['date_time']

In [ ]:
data_dir = '../input/'
fname = os.path.join(data_dir, 'climate_hour.csv')

f = open(fname)
data = f.read()
f.close()

In [ ]:
lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]

print(header)
print(len(lines))

In [ ]:
del lines[70037]

In [ ]:
import numpy as np

# Parse the data

float_data = np.zeros((len(lines), len(header) - 1))
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(',')[1:]]
    float_data[i, :] = values
print(float_data.shape)

In [ ]:
temp = float_data[:, 1]
plt.plot(range(1440), temp[:1440]);

In [ ]:
float_data[0:2]

In [ ]:
dataset[dataset['Date Time']=='31.12.2014 22:00:00'] # to find the index for split

In [ ]:
def series_to_supervised(mat,lookback,future_step=1):
    '''
    Convert a timeseries as a supervised learning dataset.
    References:
    https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/
    Arguments:
        mat: [np.ndarray] a numpy nd array object with values in float format
        lookback: [int] number of timeframes to look back in past (t-n,...,t-1). This does not include current time frame (t)
                E.g. to look 24 hours data including current time-stamp, this value should be 23 (t-23,..,t-1 and t(covered in future_step)).
        future_step: [int] number of timeframes to look in future (t,t+1,..,t+n). This includes the current time frame (t).
                E.g. to look 1 timeframe ahead in future, the value should be 2 (t and t+1).        
    Returns:
        A pandas data frame with timeseries framed for supervised learning.
    '''
    
    n_vars = mat.shape[-1]
    df = pd.DataFrame(mat)

    cols,names = list(),list()
    #create input sequence (t-n,....,t-1)
    for i in range(lookback,0,-1):
        cols.append(df.shift(i))
        names+=[('var%d(t-%d)'%(j+1,i)) for j in range(n_vars)]
    #create forecast sequence (t,t+1,..,t+n)
    for i in range(0,future_step):
        cols.append(df.shift(-i))
        if i==0:
            names+=[('var%d(t)'%(j+1)) for j in range(n_vars)]
        else:
            names+=[('var%d(t+%d)'%(j+1,i)) for j in range(n_vars)]
    #concatenate all columns
    agg = pd.concat(cols,axis=1)
    agg.columns = names
    #drop nulls
    agg.dropna(inplace=True)
    return agg

## Dividing the data into training and testing 
we will split the data from 01.01.2009 01:00:00  to 31.12.2014 22:00:00 into traing data and 31.12.2014 00:00:00 to 1.12.2016 23:00:00 into testing data

In [ ]:
lookback = 23
future_step = 2

#create train_data
train = series_to_supervised(float_data[:52564+future_step],23,2)
# Seperate the samples and target variables
x_train = train.iloc[:,:336]
y_train = train.iloc[:,337]

#create test_data
test = series_to_supervised(float_data[52565-lookback:],lookback,future_step)

# Seperate the samples and target variables
x_test = test.iloc[:,:336]
y_test = test.iloc[:,337]

### Converting the 2-D data into 3-D

In [ ]:
print("X_train data")
print(x_train.head(3))
print(x_train.tail(3))
print("\n X_test data")
print(x_test.head(3))
print(x_test.tail(3))

In [ ]:
print("Y_train data")
print(y_train.head(3))
print(y_train.tail(3))
print("\nY_test data")
print(y_test.head(3))
print(y_test.tail(3))

In [ ]:
train_X = x_train.values.reshape((x_train.shape[0],1, x_train.shape[1]))
test_X = x_test.values.reshape((x_test.shape[0],1, x_test.shape[1]))
print(train_X.shape, y_train.shape, test_X.shape, y_test.shape)

### Building baseline model using 1 layer of LSTM

In [ ]:
import time
from math import sqrt
from pandas import DataFrame
from pandas import concat

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import Dense, Dropout,Activation
from keras.layers import Embedding
from keras.layers import LSTM, GRU
from keras.layers import concatenate
from keras import optimizers

In [ ]:
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
model.summary()

In [ ]:
# fit network
model.fit(train_X, y_train, epochs=30, batch_size=72, validation_split=0.1, verbose=2, shuffle=False)

In [ ]:
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.xlabel("Epochs")
pyplot.ylabel("Loss")
pyplot.legend()
pyplot.show()

In [ ]:
predict = model.predict(test_X)

In [ ]:
columns_new = ['temperature']
predict = pd.DataFrame(predict, columns=columns_new)

In [ ]:
frames = [submission,ypred]
submission1 = pd.concat([submission,ypred], axis=1)

In [ ]:
submission1.head()

In [ ]:
submission1.to_csv('submission1.csv', index= False)

In [ ]:
'''test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
y_test = y_test.reshape((len(y_test), 1))
inv_y = concatenate((y_test, test_X[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse) '''

### Model 2

In [ ]:
model2 = Sequential()
model2.add(LSTM(512, return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])))
model2.add(Dropout(0.5))
model2.add(LSTM(512))
model2.add(Dropout(0.5))
model2.add(Dense(1))
model2.add(Activation("linear"))
model2.compile(loss="mae", optimizer="adam")
model2.summary()

In [ ]:
start = time.time()
model2.fit(train_X, y_train, batch_size=70, epochs =20,
           validation_split=0.1, verbose = 2)
print("---> Compilation Time : ", time.time() - start)

In [ ]:
pred2 = model2.predict(test_X)

In [ ]:
columns_new = ['temperature']
predict2 = pd.DataFrame(pred2, columns=columns_new)
submission2 = pd.concat([submission,predict2], axis=1)

In [ ]:
submission2.to_csv('submission2.csv', index= False)

### Model 3

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))

In [ ]:
x_train_norm = scaler.fit_transform(x_train)
x_test_norm = scaler.fit_transform(x_test)

In [ ]:
#train_X_norm = x_train_norm.values.reshape((x_train_norm[0],1,x_train_norm[1]))

In [ ]:
train_X_norm = x_train_norm.reshape((x_train_norm.shape[0],1, x_train_norm.shape[1]))
test_X_norm = x_test_norm.reshape((x_test_norm.shape[0],1, x_test_norm.shape[1]))
print(train_X_norm.shape, y_train.shape, test_X_norm.shape, y_test.shape)

In [ ]:
model3 = Sequential()
model3.add(GRU(32,dropout=0.2,recurrent_dropout=0.2,
               input_shape=(train_X_norm.shape[1],
                            train_X_norm.shape[2])))
model3.add(Dense(1))
model3.summary()

In [ ]:
model3.compile(optimizer=optimizers.RMSprop(lr=0.001), loss='MAE')

In [ ]:
model3.fit(train_X_norm, y_train, batch_size=70, epochs =20,
           validation_split=0.1, verbose = 2)

In [ ]:
pred3 = model3.predict(test_X_norm)

In [ ]:
#test_X_reg = test_X_norm.reshape((test_X_norm.shape[0], test_X_norm.shape[2]))
#test_X_reg.shape

In [ ]:
#inv_yhat = concatenate([pred3, test_X_reg], axis = 1)
#inv_yhat = scaler.inverse_transform(inv_yhat)

In [ ]:
#test_X_norm.shape

In [ ]:
columns_new = ['temperature']
predict3 = pd.DataFrame(pred3, columns=columns_new)
submission3 = pd.concat([submission,predict3], axis=1)

In [ ]:
submission3.to_csv('submission3.csv', index= False)